In [1]:
from ultralytics import YOLO
import torch
import cv2

In [2]:
torch.cuda.is_available()

True

In [3]:
#model = YOLO("yolov8l.pt")
model = YOLO("runs/detect/train2/weights/last.pt")

model.train(data="config.yaml", epochs=50, batch = 16, imgsz=480, patience=0)
model.val()

New https://pypi.org/project/ultralytics/8.0.115 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.105  Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
yolo\engine\trainer: task=detect, mode=train, model=runs/detect/train2/weights/last.pt, data=config.yaml, epochs=50, patience=0, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=Non

ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x0000021C9305BF10>
fitness: 0.8634109419991773
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.84879])
names: {0: 'iced_coffee'}
plot: True
results_dict: {'metrics/precision(B)': 0.9974401092018463, 'metrics/recall(B)': 1.0, 'metrics/mAP50(B)': 0.995, 'metrics/mAP50-95(B)': 0.8487899355546414, 'fitness': 0.8634109419991773}
save_dir: WindowsPath('runs/detect/val3')
speed: {'preprocess': 2.685448702643899, 'inference': 32.36219462226419, 'loss': 0.0, 'postprocess': 2.414605196784524}

In [4]:
best = YOLO("runs/detect/train2/weights/best.pt")
best.val()

Ultralytics YOLOv8.0.105  Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients
val: Scanning C:\Users\dell\Desktop\YOLOV8\data\val\labels.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.32s/it]
                   all         17         20       0.85          1      0.981      0.862
Speed: 2.7ms preprocess, 42.6ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs\detect\val4


ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x0000021D98322090>
fitness: 0.873582880667237
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.86168])
names: {0: 'iced_coffee'}
plot: True
results_dict: {'metrics/precision(B)': 0.8504255446450789, 'metrics/recall(B)': 1.0, 'metrics/mAP50(B)': 0.9807142857142855, 'metrics/mAP50-95(B)': 0.8616793912175649, 'fitness': 0.873582880667237}
save_dir: WindowsPath('runs/detect/val4')
speed: {'preprocess': 2.6798248291015625, 'inference': 42.6024408901439, 'loss': 0.0, 'postprocess': 1.5319235184613396}

In [5]:
# Model on img
img = cv2.imread("data/coffee.png")

results = best.predict(img, stream=True)                       # run prediction on img
for result in results:                                         # iterate results
    boxes = result.boxes.cpu().numpy()
    # get boxes on cpu in numpy
    for box in boxes:                                          # iterate boxes
        r = box.xyxy[0].astype(int)  # get corner points as int
        conf = box.conf[0]

        print(r)                                               # print boxes
        cv2.rectangle(img, r[:2], r[2:], (0, 255, 0), 2)       # draw boxes on img
        cls = result.names[int(box.cls[0])]
        label = f"{cls}_{conf:.2f}"
        cv2.putText(img, label,(r[0] - 10, r[1] - 10), 0, 0.7, (0,255,0), 2)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 288x480 4 iced_coffees, 23.0ms
Speed: 1.0ms preprocess, 23.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 480)


[413  56 531 252]
[577  90 706 298]
[456 192 589 404]
[244  95 376 287]


In [6]:
!pip install labelImg
!labelImg

In [7]:
# Model on Video
green = (0, 255, 0)

# Load the YOLO model
model = YOLO("runs/detect/train2/weights/best.pt")

# Define the class names
class_names = ['iced coffee']

# Set the path to the input video
video_path = "data/Recipe.mp4"
cap = cv2.VideoCapture(video_path)

# Get the frame dimensions and FPS of the input video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Set the output path for the annotated video
output_path = "data/output_video_2.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
output_video = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while True:
    # Read the next frame from the input video
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection on the frame using the YOLO model
    results = model.predict(frame, conf=0.25, iou=0.1)

    # Iterate over the detected objects in the frame
    for result in results:
        boxes = result.boxes.cpu().numpy()

        if len(boxes) > 0:
            xSum = 0
            ySum = 0
            for box in boxes:
                # Get the bounding box coordinates and confidence of the detected object
                r = box.xyxy[0].astype(int)
                conf = box.conf[0]

                # Calculate the center coordinates of the bounding box
                center_x = int((r[0] + r[2]) / 2)
                center_y = int((r[1] + r[3]) / 2)

                xSum += center_x
                ySum += center_y

                # Draw the bounding box and center point on the frame
                cv2.rectangle(frame, r[:2], r[2:], green, 2)
                cv2.circle(frame, (center_x, center_y), 5, green, -1)

                # Get the class label of the detected object
                cls = result.names[int(box.cls[0])]
                label = f"{cls}_{conf:.2f}"

                # Draw the class label on the frame
                cv2.putText(frame, label, (r[0] - 10, r[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


            avg_center_x = int(xSum / len(boxes))
            avg_center_y = int(ySum / len(boxes))
            cv2.circle(frame, (avg_center_x, avg_center_y), 5 , (255,0,0), -1)

            # Add the middle and mean circles
            cv2.circle(frame, (frame_width//2,frame_height//2), 5,(255,255,255),-1)
            cv2.line(frame, (avg_center_x, avg_center_y), (frame_width // 2, frame_height // 2), (0, 0, 255), 2)

            # Calculate the x and y differences
            # -x solda demek +y aşağıda demek
            diff_x = avg_center_x - frame_width // 2
            diff_y = avg_center_y - frame_height // 2
            cv2.putText(frame, f"{diff_x} {diff_y}", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


    # Write the annotated frame to the output video
    output_video.write(frame)

    # Display the annotated frame
    cv2.imshow('Webcam', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture and output video objects
cap.release()
output_video.release()

# Close all OpenCV windows
cv2.destroyAllWindows()



0: 192x320 1 iced_coffee, 64.1ms
Speed: 1.5ms preprocess, 64.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 320)

0: 192x320 1 iced_coffee, 21.0ms
Speed: 0.5ms preprocess, 21.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 320)

0: 192x320 1 iced_coffee, 12.5ms
Speed: 1.0ms preprocess, 12.5ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 320)

0: 192x320 1 iced_coffee, 13.5ms
Speed: 1.0ms preprocess, 13.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 192x320 1 iced_coffee, 12.1ms
Speed: 0.0ms preprocess, 12.1ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 320)

0: 192x320 1 iced_coffee, 12.5ms
Speed: 0.0ms preprocess, 12.5ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 320)

0: 192x320 1 iced_coffee, 12.5ms
Speed: 1.0ms preprocess, 12.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 192x320 1 iced_coffee, 13.5ms
Speed: 0.5ms preprocess, 13.5ms inference, 3.0ms 

In [8]:
# Release the video capture and output video objects
cap.release()
output_video.release()

# Close all OpenCV windows
cv2.destroyAllWindows()

In [9]:
# overlap olma durumuna bak